In [71]:
import os
import sys
import xml.etree.ElementTree as ElementTree
import collections
from collections import Counter

import psycopg2


In [2]:
racine_source = '/home/michel/legi_plat/'

In [3]:
class FrozenDict(collections.Mapping):
    """Mike Graham http://stackoverflow.com/questions/2703599/what-would-a-frozen-dict-be"""

    def __init__(self, *args, **kwargs):
        self._d = dict(*args, **kwargs)
        self._hash = None

    def __iter__(self):
        return iter(self._d)

    def __len__(self):
        return len(self._d)

    def __getitem__(self, key):
        return self._d[key]

    def __hash__(self):
        # It would have been simpler and maybe more obvious to 
        # use hash(tuple(sorted(self._d.iteritems()))) from this discussion
        # so far, but this solution is O(n). I don't know what kind of 
        # n we are going to run into, but sometimes it's hard to resist the 
        # urge to optimize when it will gain improved algorithmic performance.
        if self._hash is None:
            self._hash = 0
            for pair in self.items():
                self._hash ^= hash(pair)
        return self._hash
    
    def __repr__(self):
        return '{}({})'.format(self.__class__.__name__, str(self._d))
    
    def __str__(self):
        return self.__repr__()

In [4]:
def parse_struct(contenu):
    valeurs = {}
    
    TEXTELR = ElementTree.fromstring(contenu)
    assert TEXTELR.tag == 'TEXTELR'

    #
    META = TEXTELR[0]
    assert META.tag == 'META'

    ##
    META_COMMUN = META[0]
    assert META_COMMUN.tag == 'META_COMMUN'

    ###
    ID = META_COMMUN[0]
    assert ID.tag == 'ID'
    valeurs['ID'] = ID.text

    id_eli_present = 0
    valeurs['ID_ELI'] = ''
    if META_COMMUN[1].tag == 'ID_ELI':
        id_eli_present = 1
        ID_ELI = META_COMMUN[1]
        valeurs['ID_ELI'] = ID_ELI.text
        
    eli_alias_present = 0
    valeurs['ID_ELI_ALIAS'] = ''
    if META_COMMUN[1 + id_eli_present].tag == 'ELI_ALIAS':
        eli_alias_present = 1
        ELI_ALIAS = META_COMMUN[1 + id_eli_present]
        assert len(list(ELI_ALIAS)) == 1
        ID_ELI_ALIAS = ELI_ALIAS[0]
        assert ID_ELI_ALIAS.tag == 'ID_ELI_ALIAS'
        valeurs['ID_ELI_ALIAS'] = ID_ELI_ALIAS.text
        
    ANCIEN_ID = META_COMMUN[1 + id_eli_present + eli_alias_present]
    assert ANCIEN_ID.tag == 'ANCIEN_ID'
    valeurs['ANCIEN_ID'] = ANCIEN_ID.text

    ORIGINE = META_COMMUN[2 + id_eli_present + eli_alias_present]
    assert ORIGINE.tag == 'ORIGINE'
    assert ORIGINE.text == 'LEGI'

    URL = META_COMMUN[3 + id_eli_present + eli_alias_present]
    assert URL.tag == 'URL'
    valeurs['URL'] = URL.text

    NATURE = META_COMMUN[4 + id_eli_present + eli_alias_present]
    assert NATURE.tag == 'NATURE'
    valeurs['NATURE'] = NATURE.text

    ##
    META_SPEC = META[1]
    assert META_SPEC.tag == 'META_SPEC'

    ###
    META_TEXTE_CHRONICLE = META_SPEC[0]
    assert META_TEXTE_CHRONICLE.tag == 'META_TEXTE_CHRONICLE'

    ####
    CID = META_TEXTE_CHRONICLE[0]
    assert CID.tag == 'CID'
    valeurs['CID'] = CID.text

    NUM = META_TEXTE_CHRONICLE[1]
    assert NUM.tag == 'NUM'
    valeurs['NUM'] = NUM.text

    NUM_SEQUENCE = META_TEXTE_CHRONICLE[2]
    assert NUM_SEQUENCE.tag == 'NUM_SEQUENCE'
    valeurs['NUM_SEQUENCE'] = NUM_SEQUENCE.text

    NOR = META_TEXTE_CHRONICLE[3]
    assert NOR.tag == 'NOR'
    valeurs['NOR'] = NOR.text

    DATE_PUBLI = META_TEXTE_CHRONICLE[4]
    assert DATE_PUBLI.tag == 'DATE_PUBLI'
    valeurs['DATE_PUBLI'] = DATE_PUBLI.text

    DATE_TEXTE = META_TEXTE_CHRONICLE[5]
    assert DATE_TEXTE.tag == 'DATE_TEXTE'
    valeurs['DATE_TEXTE'] = DATE_TEXTE.text

    der_modif_present = 0
    valeurs['DERNIERE_MODIFICATION'] = ''
    if META_TEXTE_CHRONICLE[6].tag == 'DERNIERE_MODIFICATION':
        der_modif_present = 1
        DERNIERE_MODIFICATION = META_TEXTE_CHRONICLE[6]
        valeurs['DERNIERE_MODIFICATION'] = DERNIERE_MODIFICATION.text
    
    versions_a_venir_present = 0
    VERSIONS_A_VENIR_data = []
    if META_TEXTE_CHRONICLE[6 + der_modif_present].tag == 'VERSIONS_A_VENIR':
        versions_a_venir_present = 1
        VERSIONS_A_VENIR = META_TEXTE_CHRONICLE[6 + der_modif_present]
        
        #####
        for VERSION_A_VENIR in VERSIONS_A_VENIR:
            assert VERSION_A_VENIR.tag == 'VERSION_A_VENIR'
            VERSIONS_A_VENIR_data.append(VERSION_A_VENIR.text)
    valeurs['VERSIONS_A_VENIR'] = tuple(VERSIONS_A_VENIR_data)
    
    ORIGINE_PUBLI = META_TEXTE_CHRONICLE[6 + der_modif_present + versions_a_venir_present]
    assert ORIGINE_PUBLI.tag == 'ORIGINE_PUBLI'
    valeurs['ORIGINE_PUBLI'] = ORIGINE_PUBLI.text
    
    PAGE_DEB_PUBLI = META_TEXTE_CHRONICLE[7 + der_modif_present + versions_a_venir_present]
    assert PAGE_DEB_PUBLI.tag == 'PAGE_DEB_PUBLI'
    valeurs['PAGE_DEB_PUBLI'] = PAGE_DEB_PUBLI.text
    
    PAGE_FIN_PUBLI = META_TEXTE_CHRONICLE[8 + der_modif_present + versions_a_venir_present]
    assert PAGE_FIN_PUBLI.tag == 'PAGE_FIN_PUBLI'
    valeurs['PAGE_FIN_PUBLI'] = PAGE_FIN_PUBLI.text

    #
    VERSIONS = TEXTELR[1]
    assert VERSIONS.tag == 'VERSIONS'
    
    ##
    VERSIONS_data = []
    for VERSION in VERSIONS:
        assert VERSION.tag == 'VERSION'
        
        VERSION_data = {}
        
        VERSION_data['etat'] = VERSION.attrib['etat']
        
        ###
        LIEN_TXT = VERSION[0]
        assert LIEN_TXT.tag == 'LIEN_TXT'
        
        VERSION_data['debut'] = LIEN_TXT.attrib['debut']
        VERSION_data['fin'] = LIEN_TXT.attrib['fin']
        VERSION_data['id_'] = LIEN_TXT.attrib['id']
        VERSION_data['num'] = LIEN_TXT.attrib['num']
        
        VERSIONS_data.append(FrozenDict(VERSION_data))
    valeurs['VERSIONS'] = frozenset(VERSIONS_data)
        
    #
    STRUCT = TEXTELR[2]
    assert STRUCT.tag == 'STRUCT'
    
    ##
    LIENS_ART_data = []
    LIENS_SECTION_TA_data = []
    for LIEN in STRUCT:
        if LIEN.tag == 'LIEN_ART':
            LIEN_ART_data = {}

            LIEN_ART_data['debut'] = LIEN.attrib['debut']
            LIEN_ART_data['etat'] = LIEN.attrib['etat']
            LIEN_ART_data['fin'] = LIEN.attrib['fin']
            LIEN_ART_data['id_'] = LIEN.attrib['id']
            LIEN_ART_data['num'] = LIEN.attrib['num']
            LIEN_ART_data['origine'] = LIEN.attrib['origine']

            LIENS_ART_data.append(FrozenDict(LIEN_ART_data))
            
        elif LIEN.tag == 'LIEN_SECTION_TA':
            LIEN_SECTION_TA_data = {}

            LIEN_SECTION_TA_data['cid'] = LIEN.attrib['cid']
            LIEN_SECTION_TA_data['debut'] = LIEN.attrib['debut']
            LIEN_SECTION_TA_data['etat'] = LIEN.attrib['etat']
            LIEN_SECTION_TA_data['fin'] = LIEN.attrib['fin']
            LIEN_SECTION_TA_data['id_'] = LIEN.attrib['id']
            LIEN_SECTION_TA_data['niv'] = LIEN.attrib['niv']
            LIEN_SECTION_TA_data['url'] = LIEN.attrib['url']

            LIENS_SECTION_TA_data.append(FrozenDict(LIEN_SECTION_TA_data))
            
        else:
            raise ValueError(LIEN.TAG)
    valeurs['LIENS_ART'] = tuple(LIENS_ART_data)
    valeurs['LIENS_SECTION_TA'] = tuple(LIENS_SECTION_TA_data)

    return valeurs

In [5]:
def parse_version(contenu):
    valeurs = {}
    
    TEXTE_VERSION = ElementTree.fromstring(contenu)
    assert TEXTE_VERSION.tag == 'TEXTE_VERSION'

    #
    META = TEXTE_VERSION[0]
    assert META.tag == 'META'

    ##
    META_COMMUN = META[0]
    assert META_COMMUN.tag == 'META_COMMUN'

    ###
    ID = META_COMMUN[0]
    assert ID.tag == 'ID'
    valeurs['ID'] = ID.text

    id_eli_present = 0
    valeurs['ID_ELI'] = ''
    if META_COMMUN[1].tag == 'ID_ELI':
        id_eli_present = 1
        ID_ELI = META_COMMUN[1]
        valeurs['ID_ELI'] = ID_ELI.text
        
    eli_alias_present = 0
    valeurs['ID_ELI_ALIAS'] = ''
    if META_COMMUN[1 + id_eli_present].tag == 'ELI_ALIAS':
        eli_alias_present = 1
        ELI_ALIAS = META_COMMUN[1 + id_eli_present]
        assert len(list(ELI_ALIAS)) == 1
        ID_ELI_ALIAS = ELI_ALIAS[0]
        assert ID_ELI_ALIAS.tag == 'ID_ELI_ALIAS'
        valeurs['ID_ELI_ALIAS'] = ID_ELI_ALIAS.text
        
    ANCIEN_ID = META_COMMUN[1 + id_eli_present + eli_alias_present]
    assert ANCIEN_ID.tag == 'ANCIEN_ID'
    valeurs['ANCIEN_ID'] = ANCIEN_ID.text

    ORIGINE = META_COMMUN[2 + id_eli_present + eli_alias_present]
    assert ORIGINE.tag == 'ORIGINE'
    assert ORIGINE.text == 'LEGI'

    URL = META_COMMUN[3 + id_eli_present + eli_alias_present]
    assert URL.tag == 'URL'
    valeurs['URL'] = URL.text

    NATURE = META_COMMUN[4 + id_eli_present + eli_alias_present]
    assert NATURE.tag == 'NATURE'
    valeurs['NATURE'] = NATURE.text

    ##
    META_SPEC = META[1]
    assert META_SPEC.tag == 'META_SPEC'

    ###
    META_TEXTE_CHRONICLE = META_SPEC[0]
    assert META_TEXTE_CHRONICLE.tag == 'META_TEXTE_CHRONICLE'

    ####
    CID = META_TEXTE_CHRONICLE[0]
    assert CID.tag == 'CID'
    valeurs['CID'] = CID.text

    NUM = META_TEXTE_CHRONICLE[1]
    assert NUM.tag == 'NUM'
    valeurs['NUM'] = NUM.text

    NUM_SEQUENCE = META_TEXTE_CHRONICLE[2]
    assert NUM_SEQUENCE.tag == 'NUM_SEQUENCE'
    valeurs['NUM_SEQUENCE'] = NUM_SEQUENCE.text

    NOR = META_TEXTE_CHRONICLE[3]
    assert NOR.tag == 'NOR'
    valeurs['NOR'] = NOR.text

    DATE_PUBLI = META_TEXTE_CHRONICLE[4]
    assert DATE_PUBLI.tag == 'DATE_PUBLI'
    valeurs['DATE_PUBLI'] = DATE_PUBLI.text

    DATE_TEXTE = META_TEXTE_CHRONICLE[5]
    assert DATE_TEXTE.tag == 'DATE_TEXTE'
    valeurs['DATE_TEXTE'] = DATE_TEXTE.text

    der_modif_present = 0
    valeurs['DERNIERE_MODIFICATION'] = ''
    if META_TEXTE_CHRONICLE[6].tag == 'DERNIERE_MODIFICATION':
        der_modif_present = 1
        DERNIERE_MODIFICATION = META_TEXTE_CHRONICLE[6]
        valeurs['DERNIERE_MODIFICATION'] = DERNIERE_MODIFICATION.text
    
    versions_a_venir_present = 0
    VERSIONS_A_VENIR_data = []
    if META_TEXTE_CHRONICLE[6 + der_modif_present].tag == 'VERSIONS_A_VENIR':
        versions_a_venir_present = 1
        VERSIONS_A_VENIR = META_TEXTE_CHRONICLE[6 + der_modif_present]
        
        #####
        for VERSION_A_VENIR in VERSIONS_A_VENIR:
            assert VERSION_A_VENIR.tag == 'VERSION_A_VENIR'
            VERSIONS_A_VENIR_data.append(VERSION_A_VENIR.text)
    valeurs['VERSIONS_A_VENIR'] = tuple(VERSIONS_A_VENIR_data)
    
    ORIGINE_PUBLI = META_TEXTE_CHRONICLE[6 + der_modif_present + versions_a_venir_present]
    assert ORIGINE_PUBLI.tag == 'ORIGINE_PUBLI'
    valeurs['ORIGINE_PUBLI'] = ORIGINE_PUBLI.text
    
    PAGE_DEB_PUBLI = META_TEXTE_CHRONICLE[7 + der_modif_present + versions_a_venir_present]
    assert PAGE_DEB_PUBLI.tag == 'PAGE_DEB_PUBLI'
    valeurs['PAGE_DEB_PUBLI'] = PAGE_DEB_PUBLI.text
    
    PAGE_FIN_PUBLI = META_TEXTE_CHRONICLE[8 + der_modif_present + versions_a_venir_present]
    assert PAGE_FIN_PUBLI.tag == 'PAGE_FIN_PUBLI'
    valeurs['PAGE_FIN_PUBLI'] = PAGE_FIN_PUBLI.text

    ###
    META_TEXTE_VERSION = META_SPEC[1]
    assert META_TEXTE_VERSION.tag == 'META_TEXTE_VERSION'

    ####
    TITRE = META_TEXTE_VERSION[0]
    assert TITRE.tag == 'TITRE'
    valeurs['TITRE'] = TITRE.text

    TITREFULL = META_TEXTE_VERSION[1]
    assert TITREFULL.tag == 'TITREFULL'
    valeurs['TITREFULL'] = TITREFULL.text

    ETAT = META_TEXTE_VERSION[2]
    assert ETAT.tag == 'ETAT'
    valeurs['ETAT'] = ETAT.text

    DATE_DEBUT = META_TEXTE_VERSION[3]
    assert DATE_DEBUT.tag == 'DATE_DEBUT'
    valeurs['DATE_DEBUT'] = DATE_DEBUT.text

    DATE_FIN = META_TEXTE_VERSION[4]
    assert DATE_FIN.tag == 'DATE_FIN'
    valeurs['DATE_FIN'] = DATE_FIN.text

    AUTORITE = META_TEXTE_VERSION[5]
    assert AUTORITE.tag == 'AUTORITE'
    valeurs['AUTORITE'] = AUTORITE.text

    MINISTERE = META_TEXTE_VERSION[6]
    assert MINISTERE.tag == 'MINISTERE'
    valeurs['MINISTERE'] = MINISTERE.text

    liens_present = 0
    LIENS_data = []
    assert len(META_TEXTE_VERSION) in [7, 8]
    if len(META_TEXTE_VERSION) == 8:
        liens_present = 1
        LIENS = META_TEXTE_VERSION[7]
        assert LIENS.tag == 'LIENS'
        
        #####
        for LIEN in LIENS:
            assert LIEN.tag == 'LIEN'

            LIEN_data = {}

            LIEN_data['cidtexte'] = LIEN.attrib['cidtexte']
            LIEN_data['datesignatexte'] = LIEN.attrib['datesignatexte']
            LIEN_data['id_'] = LIEN.attrib['id']
            LIEN_data['naturetexte'] = LIEN.attrib['naturetexte']
            LIEN_data['nortexte'] = LIEN.attrib['nortexte']
            LIEN_data['num'] = LIEN.attrib['num']
            LIEN_data['numtexte'] = LIEN.attrib['numtexte']
            LIEN_data['sens'] = LIEN.attrib['sens']
            LIEN_data['typelien'] = LIEN.attrib['typelien']
            LIEN_data['texte'] = LIEN.text

            LIENS_data.append(FrozenDict(LIEN_data))
    valeurs['LIENS'] = tuple(LIENS_data)
      
    #
    VISAS = TEXTE_VERSION[1]
    assert VISAS.tag == 'VISAS'

    ##
    CONTENU = VISAS[0]
    valeurs['VISAS'] = ElementTree.tostring(CONTENU, encoding='unicode', method='xml')
      
    #
    SIGNATAIRES = TEXTE_VERSION[2]
    assert SIGNATAIRES.tag == 'SIGNATAIRES'

    ##
    CONTENU = SIGNATAIRES[0]
    valeurs['SIGNATAIRES'] = ElementTree.tostring(CONTENU, encoding='unicode', method='xml')
      
    #
    TP = TEXTE_VERSION[3]
    assert TP.tag == 'TP'

    ##
    CONTENU = TP[0]
    valeurs['TP'] = ElementTree.tostring(CONTENU, encoding='unicode', method='xml')
      
    #
    NOTA = TEXTE_VERSION[4]
    assert NOTA.tag == 'NOTA'

    ##
    CONTENU = NOTA[0]
    valeurs['NOTA'] = ElementTree.tostring(CONTENU, encoding='unicode', method='xml')
      
    #
    ABRO = TEXTE_VERSION[5]
    assert ABRO.tag == 'ABRO'

    ##
    CONTENU = ABRO[0]
    valeurs['ABRO'] = ElementTree.tostring(CONTENU, encoding='unicode', method='xml')
      
    #
    RECT = TEXTE_VERSION[6]
    assert RECT.tag == 'RECT'

    ##
    CONTENU = RECT[0]
    valeurs['RECT'] = ElementTree.tostring(CONTENU, encoding='unicode', method='xml')
            
    return valeurs

In [6]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

curseur.execute("select cid, categorie from texte where etat = 'valide';")
liste_texte = curseur.fetchall()

curseur.close()
connection.close()

In [7]:
def trouve_enfants(dossier):
    enfants = os.listdir(dossier)
    est_fichier_liste = [os.path.isfile(os.path.join(dossier, f)) for f in enfants]
    enfants_fichier = [enfants[i] for i in range(len(enfants)) if est_fichier_liste[i]]
    enfants_dossier = [enfants[i] for i in range(len(enfants)) if not est_fichier_liste[i]]
    
    return enfants_fichier, enfants_dossier

In [8]:
def parse_cid(categorie, cid):
    enfants_fichier, enfants_dossier = trouve_enfants(os.path.join(racine_source, categorie, cid, 'struct'))
    liste_id_ = [enfant_fichier[:20] for enfant_fichier in enfants_fichier]
    
    valeurs_struct_par_id_ = {}
    valeurs_version_par_id_ = {}
    for id_ in liste_id_:
        nom_fichier = os.path.join(racine_source, categorie, cid, 'struct', id_ + '.xml')
        with open(nom_fichier) as f:
            contenu = f.read()
        valeurs_struct = parse_struct(contenu)
        valeurs_struct_par_id_[id_] = valeurs_struct

        nom_fichier = os.path.join(racine_source, categorie, cid, 'version', id_ + '.xml')
        with open(nom_fichier) as f:
            contenu = f.read()
        valeurs_version = parse_version(contenu)
        valeurs_version_par_id_[id_] = valeurs_version

    
    return liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_

In [23]:
infos_communes = [
    'ID_ELI',
    'ID_ELI_ALIAS',
    'NATURE',
    'CID',
    'NUM',
    'NUM_SEQUENCE',
    'NOR',
    'DATE_PUBLI',
    'DATE_TEXTE',
    'DERNIERE_MODIFICATION',
    'VERSIONS_A_VENIR',
    'ORIGINE_PUBLI',
    'PAGE_DEB_PUBLI',
    'PAGE_FIN_PUBLI',
]

infos_communes_struct = [
    'VERSIONS',
    'LIENS_ART',
    'LIENS_SECTION_TA',
]

infos_communes_version = [
    'AUTORITE',
    'MINISTERE',
]

infos_particulieres = [
    'ID',
    'ANCIEN_ID',
]

infos_particulieres_struct = [
    'URL',
]

infos_particulieres_version = [
    'URL',
    'TITRE',
    'TITREFULL',
    'ETAT',    
    'DATE_DEBUT',
    'DATE_FIN',
    'LIENS',
    'VISAS',
    'SIGNATAIRES',
    'TP',
    'NOTA',
    'ABRO',
    'RECT',
]

In [24]:
cid, categorie = 'JORFTEXT000021979777', 'TNC_en_vigueur_JORF'
liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_ = parse_cid(categorie, cid)
id_ = liste_id_[0]

assert (set(valeurs_struct_par_id_[id_].keys()) ==
    set(infos_communes + infos_communes_struct + infos_particulieres + infos_particulieres_struct))

assert (set(valeurs_version_par_id_[id_].keys()) == 
    set(infos_communes + infos_communes_version + infos_particulieres + infos_particulieres_version))

## Fusionne et vérifie les informations communes au cid

In [34]:
def verifie_infos_communes(categorie, cid, liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_):
    liste_cid = ([valeurs_struct_par_id_[id_]['CID'] for id_ in liste_id_] + 
                 [valeurs_version_par_id_[id_]['CID'] for id_ in liste_id_]) 
    assert (len(set(liste_cid)) == 1) and (liste_cid[0] == cid), ('{} {} {} {}'.format(categorie, cid, 'CID', set(liste_cid)))
    
    infos_communes_texte = {}
    for info_commune in infos_communes + infos_communes_struct + infos_communes_version:
        if info_commune in infos_communes:
            valeurs = ([valeurs_struct_par_id_[id_][info_commune] for id_ in liste_id_] + 
                       [valeurs_version_par_id_[id_][info_commune] for id_ in liste_id_])
        elif info_commune in infos_communes_struct:
            valeurs = [valeurs_struct_par_id_[id_][info_commune] for id_ in liste_id_]
        elif info_commune in infos_communes_version:
            valeurs = [valeurs_version_par_id_[id_][info_commune] for id_ in liste_id_]
            
        valeurs_non_nulles = list(set(valeurs) - {None})
        assert len(valeurs_non_nulles) <= 1, ('{} {} {} {}'.format(categorie, cid, info_commune, set(valeurs)))
        infos_communes_texte[info_commune] = valeurs_non_nulles[0] if len(valeurs_non_nulles) >= 1 else None

    return infos_communes_texte

In [35]:
liste_erreurs = []
infos_communes_par_cid = {}
for cid, categorie in liste_texte:
    liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_ = parse_cid(categorie, cid)
    try:
        infos_communes_par_cid[cid] = verifie_infos_communes(categorie, cid, liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_)
    except AssertionError as e:
        print(e)
        liste_erreurs.append(e)

Report du 1er passage (64 erreurs) :
```
TNC_non_vigueur_JORF JORFTEXT000000644046 DATE_PUBLI {'2006-09-03', '2006-09-06'}
TNC_en_vigueur_JORF JORFTEXT000000841628 DATE_TEXTE {'1991-08-21', '1991-07-20'}
TNC_en_vigueur_JORF JORFTEXT000000763720 DATE_PUBLI {'2001-12-21', '2001-12-25'}
TNC_en_vigueur_JORF JORFTEXT000000392850 DATE_TEXTE {'1998-03-16', '1998-12-22'}
TNC_en_vigueur_JORF JORFTEXT000000844102 DATE_TEXTE {'1987-06-10', '1987-06-01'}
TNC_en_vigueur_JORF JORFTEXT000000225685 DATE_PUBLI {'2001-11-09', '2001-11-08'}
TNC_en_vigueur_JORF JORFTEXT000000174965 DATE_PUBLI {'1992-07-06', '1992-07-03'}
TNC_non_vigueur_JORF JORFTEXT000000553295 DATE_PUBLI {'1998-02-15', '1998-02-14'}
TNC_en_vigueur_JORF JORFTEXT000000593204 DATE_PUBLI {'2002-01-06', '2002-01-07'}
TNC_en_vigueur_JORF JORFTEXT000000510005 NATURE {'LOI_ORGANIQUE', 'LOI'}
TNC_en_vigueur_JORF JORFTEXT000000244335 DATE_PUBLI {'2007-04-21', '2007-04-22'}
TNC_en_vigueur_JORF JORFTEXT000000408052 DATE_PUBLI {'2001-11-10', '2001-11-09'}
TNC_non_vigueur_JORF JORFTEXT000018699380 DATE_PUBLI {'2008-04-27', '2008-04-29'}
TNC_en_vigueur_JORF JORFTEXT000020506850 DATE_PUBLI {'2009-04-12', '2009-04-14'}
TNC_en_vigueur_JORF JORFTEXT000000759488 DATE_PUBLI {'1999-04-03', '1999-04-04'}
TNC_en_vigueur_JORF JORFTEXT000000499751 DATE_TEXTE {'1989-12-14', '1978-01-13'}
TNC_non_vigueur_JORF JORFTEXT000000447508 DATE_PUBLI {'2005-08-07', '2005-08-06'}
TNC_non_vigueur_JORF JORFTEXT000000424033 DATE_PUBLI {'2005-05-07', '2005-05-12'}
TNC_en_vigueur_JORF JORFTEXT000000412879 DATE_PUBLI {'2002-05-15', '2002-05-05'}
TNC_en_vigueur_JORF JORFTEXT000020473987 DATE_PUBLI {'2009-04-04', '2009-04-05'}
TNC_en_vigueur_JORF JORFTEXT000020795196 DATE_PUBLI {'1939-02-05', '1939-01-19'}
TNC_non_vigueur_JORF JORFTEXT000000264768 DATE_PUBLI {'2006-04-10', '2006-04-11'}
TNC_en_vigueur_JORF JORFTEXT000000418078 DATE_PUBLI {'2003-02-06', '2003-02-08'}
TNC_en_vigueur_JORF JORFTEXT000000468155 DATE_PUBLI {'1984-08-28', '1984-08-29'}
TNC_en_vigueur_JORF JORFTEXT000000598735 DATE_PUBLI {'2002-11-23', '2002-11-25'}
TNC_en_vigueur_JORF JORFTEXT000000590733 DATE_PUBLI {'2001-10-15', '2001-10-13'}
TNC_en_vigueur_JORF JORFTEXT000000250830 DATE_PUBLI {'2004-08-04', '2004-07-31'}
TNC_en_vigueur_JORF JORFTEXT000000226802 DATE_PUBLI {'2002-02-13', '2002-02-07'}
TNC_non_vigueur_JORF JORFTEXT000019025755 DATE_PUBLI {'2008-06-21', '2008-06-20'}
TNC_non_vigueur_JORF JORFTEXT000000468097 DATE_PUBLI {'1983-06-18', '1983-06-21'}
TNC_en_vigueur_JORF JORFTEXT000000403567 DATE_PUBLI {'2002-03-29', '2002-03-25'}
TNC_en_vigueur_JORF JORFTEXT000000819750 DATE_PUBLI {'2006-08-08', '2006-08-13'}
TNC_en_vigueur_JORF JORFTEXT000000367879 NUM {'96-64', '94-1098'}
TNC_en_vigueur_JORF JORFTEXT000018838055 DATE_PUBLI {'2008-05-29', '2008-06-02'}
TNC_non_vigueur_JORF JORFTEXT000000355142 DATE_PUBLI {'1991-06-14', '1991-06-10'}
TNC_non_vigueur_JORF JORFTEXT000000265798 DATE_PUBLI {'2006-01-27', '2006-01-28'}
TNC_en_vigueur_JORF JORFTEXT000018838015 DATE_PUBLI {'2008-05-24', '2008-05-28'}
TNC_en_vigueur_JORF JORFTEXT000019375864 DATE_PUBLI {'2008-09-11', '2008-09-10'}
TNC_non_vigueur_JORF JORFTEXT000000542086 DATE_PUBLI {'1992-08-07', '1992-08-08'}
TNC_non_vigueur_JORF JORFTEXT000000811401 DATE_PUBLI {'2006-01-01', '2005-12-31'}
TNC_non_vigueur_JORF JORFTEXT000018700167 DATE_PUBLI {'2008-05-03', '2008-05-02'}
TNC_non_vigueur_JORF JORFTEXT000020568544 DATE_PUBLI {'2009-05-13', '2009-05-03'}
TNC_en_vigueur_JORF JORFTEXT000000777708 DATE_PUBLI {'2003-01-14', '2003-01-11'}
TNC_en_vigueur_JORF JORFTEXT000000213264 DATE_PUBLI {'1999-12-11', '1999-12-10'}
TNC_en_vigueur_JORF JORFTEXT000000233691 DATE_PUBLI {'2004-05-10', '2004-05-11'}
TNC_en_vigueur_JORF JORFTEXT000000327186 DATE_TEXTE {'1991-10-31', '1991-12-18'}
TNC_en_vigueur_JORF JORFTEXT000020128976 DATE_PUBLI {'2009-01-22', '2009-01-23'}
TNC_en_vigueur_JORF JORFTEXT000000755111 DATE_PUBLI {'2000-12-23', '2000-12-22'}
TNC_non_vigueur_JORF JORFTEXT000000336642 DATE_PUBLI {'1972-01-16', '1972-01-14'}
TNC_en_vigueur_JORF JORFTEXT000000821801 DATE_PUBLI {'2006-12-30', '2007-01-01'}
TNC_non_vigueur_JORF JORFTEXT000000321578 DATE_TEXTE {'1987-03-19', '1980-04-28'}
TNC_en_vigueur_JORF JORFTEXT000019917301 DATE_PUBLI {'2008-12-17', '2008-12-14'}
TNC_non_vigueur_JORF JORFTEXT000000591402 DATE_PUBLI {'2001-10-21', '2001-10-18'}
TNC_en_vigueur_JORF JORFTEXT000000332963 NUM {'87-801', '87-802'}
TNC_non_vigueur_JORF JORFTEXT000000811398 DATE_PUBLI {'2006-01-01', '2005-12-31'}
TNC_en_vigueur_JORF JORFTEXT000000256844 DATE_PUBLI {'2004-10-15', '2004-10-16'}
TNC_en_vigueur_JORF JORFTEXT000000415961 DATE_PUBLI {'2002-09-25', '2002-09-24'}
TNC_non_vigueur_JORF JORFTEXT000019341856 DATE_PUBLI {'2008-09-05', '2008-09-06'}
TNC_non_vigueur_JORF JORFTEXT000000391926 DATE_PUBLI {'2001-03-12', '2001-03-15'}
TNC_en_vigueur_JORF JORFTEXT000000244450 DATE_PUBLI {'2003-11-12', '2003-11-15'}
TNC_en_vigueur_JORF JORFTEXT000000247419 DATE_PUBLI {'2004-02-22', '2004-02-26'}
TNC_en_vigueur_JORF JORFTEXT000000465978 DATE_PUBLI {'2007-03-13', '2007-03-14'}
TNC_en_vigueur_JORF JORFTEXT000000772680 DATE_PUBLI {'2001-09-19', '2001-09-18'}
TNC_en_vigueur_JORF JORFTEXT000000316622 DATE_TEXTE {'1991-04-12', '1991-03-29'}
```

In [36]:
len(liste_erreurs)

0

In [14]:
liste_cid_anomalie = [str(erreur).split(' ')[1] for erreur in liste_erreurs]
', '.join(liste_cid_anomalie)

''

Ajout de la colonne `champs_communs_coherents` :
```
alter table texte add champs_communs_coherents boolean;
```

In [211]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

for cid, categorie in liste_texte:
    curseur.execute("select etat from texte where cid = %s;", (cid,))
    resultats_requete = curseur.fetchall()
    assert len(resultats_requete) == 1
    etat_vieux = resultats_requete[0][0]
    
    champs_communs_coherents = (cid not in liste_cid_anomalie)
    
    etat = 'valide' if ((etat_vieux == 'valide') and champs_communs_coherents) else 'anomalie'
    
    curseur.execute("update texte set champs_communs_coherents = %s, etat = %s where cid = %s;",
                    (champs_communs_coherents, etat, cid))
    connection.commit()
    
curseur.close()
connection.close()

95599 textes valides

## Vérifie les informations particulières à une version

In [52]:
def verifie_infos_particulieres(categorie, cid, liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_):
    for info_particuliere in infos_particulieres:
        for id_ in liste_id_:
            assert id_ == valeurs_version_par_id_[id_]['ID']
            
            valeur_struct = valeurs_struct_par_id_[id_][info_particuliere]
            valeur_version = valeurs_version_par_id_[id_][info_particuliere]
            
            assert valeur_struct == valeur_version, ('{} {} {} {} != {}'.format(
                categorie, cid, info_particuliere, valeur_struct, valeur_version))

In [53]:
liste_erreurs = []
for cid, categorie in liste_texte:
    liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_ = parse_cid(categorie, cid)
    try:
        verifie_infos_particulieres(categorie, cid, liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_)
    except AssertionError as e:
        print(e)
        liste_erreurs.append(e)

In [54]:
len(liste_erreurs)

0

## Unicité des id\_ (un même id\_ n'apparaît que pour un seul cid)

In [62]:
liste_id_par_cid = {}
for cid, categorie in liste_texte:
    enfants_fichier, enfants_dossier = trouve_enfants(os.path.join(racine_source, categorie, cid, 'struct'))
    liste_id_ = [enfant_fichier[:20] for enfant_fichier in enfants_fichier]

    liste_id_par_cid[cid] = liste_id_

In [68]:
liste_id_ = [id_ for cid, l in liste_id_par_cid.items() for id_ in l]
set_id_ = set(liste_id_)

In [69]:
len(liste_id_), len(set_id_)  # 1er passage : (97882, 97881)

(97882, 97881)

In [74]:
Counter(Counter(liste_id_).values())  # 1er passage : Counter({1: 97880, 2: 1})

Counter({1: 97880, 2: 1})

In [81]:
dict_id_duplication = {id_: nb for id_, nb in Counter(liste_id_).items() if nb > 1}

In [82]:
dict_id_duplication  # 1er passage : {'LEGITEXT000006077178': 2}

{'LEGITEXT000006077178': 2}

In [85]:
liste_id_duplication = list(dict_id_duplication.keys())
ensemble_id_duplication = set(liste_id_duplication)
liste_id_duplication  # 1er passage : ['LEGITEXT000006077178']

['LEGITEXT000006077178']

In [87]:
liste_cid_duplication = [cid for cid, l in liste_id_par_cid.items() if ensemble_id_duplication & set(l)]

In [89]:
liste_cid_duplication  # 1er passage : ['JORFTEXT000000343573', 'JORFTEXT000000160046']

['JORFTEXT000000343573', 'JORFTEXT000000160046']

Ajout de la colonne `non_duplication_id_` :
```
alter table texte add non_duplication_id_ boolean;
```

In [ ]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

for cid, categorie in liste_texte:
    curseur.execute("select etat from texte where cid = %s;", (cid,))
    resultats_requete = curseur.fetchall()
    assert len(resultats_requete) == 1
    etat_vieux = resultats_requete[0][0]
    
    non_duplication_id_ = (cid not in liste_cid_duplication)
    
    etat = 'valide' if ((etat_vieux == 'valide') and non_duplication_id_) else 'anomalie'
    
    curseur.execute("update texte set non_duplication_id_ = %s, etat = %s where cid = %s;",
                    (non_duplication_id_, etat, cid))
    connection.commit()
    
curseur.close()
connection.close()

95597 textes valides

## Regroupe les informations particulières à une version

In [56]:
def regroupe_infos_particulieres(categorie, cid, id_, valeurs_struct_par_id_, valeurs_version_par_id_):
    info_particuliere_version = {
        'id_': id_,
        'cid': cid,
        'categorie': categorie,
        'ANCIEN_ID': valeurs_version_par_id_[id_]['ANCIEN_ID'],
        'TITRE': valeurs_version_par_id_[id_]['TITRE'],
        'TITREFULL': valeurs_version_par_id_[id_]['TITREFULL'],
        'ETAT': valeurs_version_par_id_[id_]['ETAT'],
        'DATE_DEBUT': valeurs_version_par_id_[id_]['DATE_DEBUT'],
        'DATE_FIN': valeurs_version_par_id_[id_]['DATE_FIN'],
        'VISAS': valeurs_version_par_id_[id_]['VISAS'],
        'SIGNATAIRES': valeurs_version_par_id_[id_]['SIGNATAIRES'],
        'TP': valeurs_version_par_id_[id_]['TP'],
        'NOTA': valeurs_version_par_id_[id_]['NOTA'],
        'ABRO': valeurs_version_par_id_[id_]['ABRO'],
        'RECT': valeurs_version_par_id_[id_]['RECT'],
    }
    
    return info_particuliere_version


In [60]:
infos_particulieres_par_id_ = {}
for cid, categorie in liste_texte:
    liste_id_, valeurs_struct_par_id_, valeurs_version_par_id_ = parse_cid(categorie, cid)
    
    for id_ in liste_id_:
        assert id_ not in infos_particulieres_par_id_, (categorie, cid, id_)
        infos_particulieres_par_id_[id_] = regroupe_infos_particulieres(categorie, cid, id_, valeurs_struct_par_id_, valeurs_version_par_id_)


In [ ]:
LIENS

In [61]:
infos_particulieres_par_id_

{'JORFTEXT000000248184': ['LEGITEXT000005689685'],
 'JORFTEXT000018885616': ['LEGITEXT000018892160'],
 'JORFTEXT000031131779': ['LEGITEXT000031132560'],
 'JORFTEXT000000884126': ['LEGITEXT000006062695'],
 'JORFTEXT000000592555': ['LEGITEXT000005631982'],
 'JORFTEXT000000494747': ['LEGITEXT000006057937'],
 'JORFTEXT000024537026': ['LEGITEXT000024539531'],
 'JORFTEXT000029750455': ['LEGITEXT000029750995'],
 'JORFTEXT000000539232': ['LEGITEXT000006078982'],
 'JORFTEXT000020125039': ['LEGITEXT000020795043'],
 'JORFTEXT000000200080': ['LEGITEXT000005623647'],
 'JORFTEXT000021865458': ['LEGITEXT000021866862'],
 'JORFTEXT000000737533': ['LEGITEXT000005619460'],
 'JORFTEXT000000745366': ['LEGITEXT000005621655'],
 'JORFTEXT000021494577': ['LEGITEXT000021502842'],
 'JORFTEXT000026830055': ['LEGITEXT000026831653'],
 'JORFTEXT000000853775': ['LEGITEXT000006070701'],
 'JORFTEXT000000271966': ['LEGITEXT000006054280'],
 'JORFTEXT000000662632': ['LEGITEXT000006058408'],
 'JORFTEXT000020617176': ['LEGI

## Vérification de la liste des versions

In [ ]:
VERSIONS

In [49]:
infos_communes_par_cid[cid]['VERSIONS']

frozenset({FrozenDict({'debut': '2999-01-01', 'fin': '2999-01-01', 'id_': 'JORFTEXT000019712172', 'etat': '', 'num': '2008-1116'}),
           FrozenDict({'debut': '2008-11-02', 'fin': '2999-01-01', 'id_': 'LEGITEXT000019713196', 'etat': 'VIGUEUR', 'num': '2008-1116'})})

In [47]:
cid, categorie = liste_texte[0]

## Mise en base des informations communes à un cid

Ajout des nouvelles colonnes :
```
alter table texte
    add ID_ELI text,
    add ID_ELI_ALIAS text,
    add NATURE text,
    add NUM text,
    add NUM_SEQUENCE text,
    add NOR text,
    add DATE_PUBLI text,
    add DATE_TEXTE text,
    add DERNIERE_MODIFICATION text,
    add ORIGINE_PUBLI text,
    add PAGE_DEB_PUBLI text,
    add PAGE_FIN_PUBLI text,
    add AUTORITE text,
    add MINISTERE text,
;
```


In [ ]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

for cid, categorie in liste_texte:
    infos = infos_communes_par_cid[cid]
    valeurs = {
        'ID_ELI': infos_communes_par_cid['ID_ELI'],
        'ID_ELI_ALIAS': infos_communes_par_cid['ID_ELI_ALIAS'],
        'NATURE': infos_communes_par_cid['NATURE'],
        'NUM': infos_communes_par_cid['NUM'],
        'NUM_SEQUENCE': infos_communes_par_cid['NUM_SEQUENCE'],
        'NOR': infos_communes_par_cid['NOR'],
        'DATE_PUBLI': infos_communes_par_cid['DATE_PUBLI'],
        'DATE_TEXTE': infos_communes_par_cid['DATE_TEXTE'],
        'DERNIERE_MODIFICATION': infos_communes_par_cid['DERNIERE_MODIFICATION'],
        'ORIGINE_PUBLI': infos_communes_par_cid['ORIGINE_PUBLI'],
        'PAGE_DEB_PUBLI': infos_communes_par_cid['PAGE_DEB_PUBLI'],
        'PAGE_FIN_PUBLI': infos_communes_par_cid['PAGE_FIN_PUBLI'],
        'AUTORITE': infos_communes_par_cid['AUTORITE'],
        'MINISTERE': infos_communes_par_cid['MINISTERE'],
        'cid': cid,
    }

    curseur.execute("""
        update article set
            ID_ELI = %(ID_ELI)s,
            ID_ELI_ALIAS = %(ID_ELI_ALIAS)s,
            NATURE = %(NATURE)s,
            NUM = %(NUM)s,
            NUM_SEQUENCE = %(NUM_SEQUENCE)s,
            NOR = %(NOR)s,
            DATE_PUBLI = %(DATE_PUBLI)s,
            DATE_TEXTE = %(DATE_TEXTE)s,
            DERNIERE_MODIFICATION = %(DERNIERE_MODIFICATION)s,
            ORIGINE_PUBLI = %(ORIGINE_PUBLI)s,
            PAGE_DEB_PUBLI = %(PAGE_DEB_PUBLI)s,
            PAGE_FIN_PUBLI = %(PAGE_FIN_PUBLI)s,
            AUTORITE = %(AUTORITE)s,
            MINISTERE = %(MINISTERE)s,
            where cid = %(cid)s;
        """, valeurs)
    connection.commit()
    
curseur.close()
connection.close()

## Mise en base des informations particulières à une version

In [ ]:
VERSIONS

## Mise en base des liens entre une version et une autre version

## Mise en base des liens entre un texte et un article

In [ ]:
LIENS_ART

## Mise en base des liens entre un texte et une section_ta

In [ ]:
LIENS_SECTION_TA

## Mise en base des versions futures d'un texte

In [ ]:
VERSIONS_A_VENIR

In [45]:
[infos_communes_par_cid[cid]['VERSIONS_A_VENIR'] for cid, categorie in liste_texte]

[(),
 ('2015-11-01',),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 ('2016-08-07',),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 ('2017-01-01',),
 (),
 (),
 (),
 (),
 (),
 (),
 